In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
state='colorado'
today_date = 'feb8'

In [3]:
drop_years = [2017,2018,2019]

In [6]:
total_emissions = pd.read_csv('../Evolved/my outputs/{}_transportation_totalEmissions_UPDATED_dec15.csv'.format(state))
total_emissions.drop(columns='Unnamed: 0', inplace=True)
drop_year_inds = total_emissions['YEAR'].isin(drop_years)
total_emissions = total_emissions[~drop_year_inds]
total_emissions['Percentage change in emissions'] = total_emissions.groupby(['SCENARIO','pollutant'])['emissions (metric tons)'].apply(lambda x: x.div(x.iloc[0]).subtract(1))
total_emissions.to_csv('../Evolved/my outputs/{}_transportation_totalEmissions_UPDATED_PCT_{}.csv'.format(state, today_date), float_format='%.10f')

In [7]:
# Use block of code below if Nevada or Colorado.

# total_emissions_old = pd.read_csv('Evolved/my outputs/{}_transportation_totalEmissions_bySubsector_withSOx_oct7.csv'.format(state))
# total_emissions_old.drop(columns='Unnamed: 0', inplace=True)
# drop_year_inds = total_emissions_old['YEAR'].isin(drop_years)
# total_emissions_old = total_emissions_old[~drop_year_inds]
# total_emissions_old = total_emissions_old.groupby(['SCENARIO','pollutant','YEAR'])['emissions (metric tons)'].sum().reset_index()
# total_emissions_old['Percentage change in emissions'] = total_emissions_old.groupby(['SCENARIO','pollutant'])['emissions (metric tons)'].apply(lambda x: x.div(x.iloc[0]).subtract(1))
# total_emissions_old.to_csv('Evolved/my outputs/{}_transportation_totalEmissions_PCT_{}.csv'.format(state, today_date), float_format='%.10f')

In [8]:
#Use block of code below for New Mexico.

total_emissions_old = pd.read_csv('../Evolved/my outputs/{}_transportation_totalEmissions_sep14.csv'.format(state))
total_emissions_old.drop(columns='Unnamed: 0', inplace=True)
drop_year_inds = total_emissions_old['YEAR'].isin(drop_years)
total_emissions_old = total_emissions_old[~drop_year_inds]
total_emissions_old = total_emissions_old.groupby(['SCENARIO','pollutant','YEAR'])['emissions (metric tons)'].sum().reset_index()
total_emissions_old['Percentage change in emissions'] = total_emissions_old.groupby(['SCENARIO','pollutant'])['emissions (metric tons)'].apply(lambda x: x.div(x.iloc[0]).subtract(1))
total_emissions_old.to_csv('../Evolved/my outputs/{}_transportation_totalEmissions_PCT_{}.csv'.format(state, today_date), float_format='%.10f')


In [4]:
tract_area = pd.read_csv('../Demographics/demographics_tract.csv')
tract_area.drop(tract_area.columns.difference(['GEOID','tract_area']), 1, inplace=True)
tract_area.rename(columns={'GEOID':'Census Tract', 'tract_area':'Tract Area (sqmi)'},inplace=True)
tract_area.set_index('Census Tract', inplace=True)

In [5]:
subsector_emissions = pd.read_csv('../Evolved/my outputs/colorado_transportation_tractEmissions_bySubsector_UPDATED_nov23.csv'.format(state))
subsector_emissions.drop(columns='Unnamed: 0', inplace=True)

In [6]:
lda_inds = subsector_emissions['Source Type']=='light duty autos'
ldt_inds = subsector_emissions['Source Type']=='light duty trucks'
ldv_subsector_emissions = subsector_emissions[lda_inds | ldt_inds]

In [7]:
hdt_inds = subsector_emissions['Source Type']=='heavy duty trucks'
mdt_inds = subsector_emissions['Source Type']=='medium duty trucks'
hdv_subsector_emissions = subsector_emissions[hdt_inds | mdt_inds]

In [8]:
hdv_subsector_emissions

,Year,State,County,Census Tract,Sector,Source Type,Data Category,Scenario,Pollutant Category,Pollutant Type,Emissions (tonnes / year),Data source
0,2017,CO,Adams County,8001007801,Transportation,heavy duty trucks,NP,100 PERCENT RENEWABLES,CAP,CO,0.075932,Evolved
1,2017,CO,Adams County,8001007802,Transportation,heavy duty trucks,NP,100 PERCENT RENEWABLES,CAP,CO,0.183450,Evolved
2,2017,CO,Adams County,8001007900,Transportation,heavy duty trucks,NP,100 PERCENT RENEWABLES,CAP,CO,0.133279,Evolved
3,2017,CO,Adams County,8001008000,Transportation,heavy duty trucks,NP,100 PERCENT RENEWABLES,CAP,CO,0.258059,Evolved
4,2017,CO,Adams County,8001008100,Transportation,heavy duty trucks,NP,100 PERCENT RENEWABLES,CAP,CO,2.042270,Evolved
...,...,...,...,...,...,...,...,...,...,...,...,...
7670203,2050,CO,Weld County,8123002300,Transportation,medium duty trucks,NP,SLOW ELECTRICITY,GHG,CO2,34.737413,Evolved
7670204,2050,CO,Weld County,8123002501,Transportation,medium duty trucks,NP,SLOW ELECTRICITY,GHG,CO2,99.109292,Evolved
7670205,2050,CO,Weld County,8123002502,Transportation,medium duty trucks,NP,SLOW ELECTRICITY,GHG,CO2,107.436337,Evolved
7670206,2050,CO,Yuma County,8125963100,Transportation,medium duty trucks,NP,SLOW ELECTRICITY,GHG,CO2,37.328383,Evolved


In [10]:
inds2020 = subsector_emissions['Year']==2050
inds2030 = subsector_emissions['Year']==2040
pm25_inds = subsector_emissions['Pollutant Type']=='PM2.5'
central_inds = subsector_emissions['Scenario']=='CENTRAL'
ref_inds = subsector_emissions['Scenario']=='REFERENCE'
subsector_emissions_pm = subsector_emissions[inds2020 | inds2030]
subsector_emissions_pm = subsector_emissions_pm[pm25_inds]
subsector_emissions_pm = subsector_emissions_pm[central_inds | ref_inds]
grouped = subsector_emissions_pm.groupby(['Year','State','Sector','Scenario','Pollutant Category','Pollutant Type'])['Emissions (tonnes / year)'].sum().reset_index()
grouped
yeartotal = grouped.groupby(['Year'])['Emissions (tonnes / year)'].sum().reset_index()

<ipython-input-10-203ee501374c>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subsector_emissions_pm = subsector_emissions_pm[pm25_inds]
<ipython-input-10-203ee501374c>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subsector_emissions_pm = subsector_emissions_pm[central_inds | ref_inds]


### Aggregate CAPs emissions at tract level and calculate cumulative emission reductions.

In [9]:
tract_emissions = subsector_emissions.groupby(['Year','State','County',
                                               'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                                            'Data source'])['Emissions (tonnes / year)'].sum().reset_index()

tract_emissions['Baseline emissions (tonnes / year)'] = tract_emissions.groupby(['State','County',
                            'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].transform(lambda x: x.iloc[0])

tract_emissions['Reference emissions (tonnes / year)'] = tract_emissions.groupby(['State','County',
                            'Census Tract','Sector','Data Category','Pollutant Category','Pollutant Type','Year',
                            'Data source'])['Emissions (tonnes / year)'].transform(lambda x: x.iloc[3])

baseline_inds = tract_emissions['Year']==2017
tract_emissions_baseline = tract_emissions[baseline_inds]
tract_emissions_baseline_area = tract_emissions_baseline.join(tract_area, on='Census Tract')
tract_emissions_baseline_area['Emissions (Tonnes / Square Mile)'] = tract_emissions_baseline_area['Emissions (tonnes / year)'] / tract_emissions_baseline_area['Tract Area (sqmi)']
tract_emissions_baseline_area['Baseline Emissions Percentile'] = tract_emissions_baseline_area.groupby(['Scenario','Year','Pollutant Type'])['Emissions (Tonnes / Square Mile)'].rank(pct=True)

ptile_inds = tract_emissions_baseline_area['Baseline Emissions Percentile']==1
tract_emissions_baseline_area.loc[ptile_inds,'Baseline Emissions Percentile']=.999

#tract_emissions_baseline_area.to_csv('../Evolved/my outputs/{}_transportation_tractEmissions_baseline2017_{}.csv'.format(state,today_date))

In [10]:
ldv_tract_emissions = ldv_subsector_emissions.groupby(['Year','State','County',
                                               'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                                            'Data source'])['Emissions (tonnes / year)'].sum().reset_index()

In [11]:
hdv_tract_emissions = hdv_subsector_emissions.groupby(['Year','State','County',
                                               'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                                            'Data source'])['Emissions (tonnes / year)'].sum().reset_index()

In [12]:
ldv_tract_emissions['Baseline LDV emissions (tonnes / year)'] = ldv_tract_emissions.groupby(['State','County',
                            'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].transform(lambda x: x.iloc[0])

ldv_tract_emissions['Reference LDV emissions (tonnes / year)'] = ldv_tract_emissions.groupby(['State','County',
                            'Census Tract','Sector','Data Category','Pollutant Category','Pollutant Type','Year',
                            'Data source'])['Emissions (tonnes / year)'].transform(lambda x: x.iloc[3])

ldv_baseline_inds = ldv_tract_emissions['Year']==2017
ldv_tract_emissions_baseline = ldv_tract_emissions[ldv_baseline_inds]
ldv_tract_emissions_baseline_area = ldv_tract_emissions_baseline.join(tract_area, on='Census Tract')
ldv_tract_emissions_baseline_area['Emissions (Tonnes / Square Mile)'] = ldv_tract_emissions_baseline_area['Emissions (tonnes / year)'] / ldv_tract_emissions_baseline_area['Tract Area (sqmi)']
ldv_tract_emissions_baseline_area['Baseline Emissions Percentile'] = ldv_tract_emissions_baseline_area.groupby(['Scenario','Year','Pollutant Type'])['Emissions (Tonnes / Square Mile)'].rank(pct=True)

ldv_ptile_inds = ldv_tract_emissions_baseline_area['Baseline Emissions Percentile']==1
ldv_tract_emissions_baseline_area.loc[ldv_ptile_inds,'Baseline Emissions Percentile']=.999

#ldv_tract_emissions_baseline_area.to_csv('../Evolved/my outputs/{}_transportation_tractLDVEmissions_baseline2017_{}.csv'.format(state,today_date))

In [14]:
hdv_tract_emissions['Baseline HDV emissions (tonnes / year)'] = hdv_tract_emissions.groupby(['State','County',
                            'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].transform(lambda x: x.iloc[0])

hdv_tract_emissions['Reference HDV emissions (tonnes / year)'] = hdv_tract_emissions.groupby(['State','County',
                            'Census Tract','Sector','Data Category','Pollutant Category','Pollutant Type','Year',
                            'Data source'])['Emissions (tonnes / year)'].transform(lambda x: x.iloc[3])

hdv_baseline_inds = hdv_tract_emissions['Year']==2017
hdv_tract_emissions_baseline = hdv_tract_emissions[hdv_baseline_inds]
hdv_tract_emissions_baseline_area = hdv_tract_emissions_baseline.join(tract_area, on='Census Tract')
hdv_tract_emissions_baseline_area['Emissions (Tonnes / Square Mile)'] = hdv_tract_emissions_baseline_area['Emissions (tonnes / year)'] / hdv_tract_emissions_baseline_area['Tract Area (sqmi)']
hdv_tract_emissions_baseline_area['Baseline Emissions Percentile'] = hdv_tract_emissions_baseline_area.groupby(['Scenario','Year','Pollutant Type'])['Emissions (Tonnes / Square Mile)'].rank(pct=True)

hdv_ptile_inds = hdv_tract_emissions_baseline_area['Baseline Emissions Percentile']==1
hdv_tract_emissions_baseline_area.loc[hdv_ptile_inds,'Baseline Emissions Percentile']=.999

In [16]:
hdv_tract_emissions_baseline_area.to_csv('../Evolved/my outputs/{}_transportation_tractHDVEmissions_baseline2017_{}.csv'.format(state,today_date))

In [17]:
hdv_tract_emissions_baseline_area

,Year,State,County,Census Tract,Sector,Data Category,Scenario,Pollutant Category,Pollutant Type,Data source,Emissions (tonnes / year),Baseline HDV emissions (tonnes / year),Reference HDV emissions (tonnes / year),Tract Area (sqmi),Emissions (Tonnes / Square Mile),Baseline Emissions Percentile
0,2017,CO,Adams County,8001007801,Transportation,NP,100 PERCENT RENEWABLES,CAP,CO,Evolved,0.875398,0.875398,0.875735,0.247702,3.534083,0.723558
1,2017,CO,Adams County,8001007801,Transportation,NP,100 PERCENT RENEWABLES,CAP,NOx,Evolved,0.539631,0.539631,0.539742,0.247702,2.178550,0.552885
2,2017,CO,Adams County,8001007801,Transportation,NP,100 PERCENT RENEWABLES,CAP,PM10,Evolved,0.029606,0.029606,0.029607,0.247702,0.119524,0.581731
3,2017,CO,Adams County,8001007801,Transportation,NP,100 PERCENT RENEWABLES,CAP,PM2.5,Evolved,0.019352,0.019352,0.019353,0.247702,0.078124,0.580929
4,2017,CO,Adams County,8001007801,Transportation,NP,100 PERCENT RENEWABLES,CAP (precursor),VOC,Evolved,0.082180,0.082180,0.082197,0.247702,0.331771,0.661859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31195,2017,CO,Yuma County,8125963200,Transportation,NP,SLOW ELECTRICITY,CAP,CO,Evolved,34.139950,34.139950,34.140558,1034.586811,0.032999,0.054487
31196,2017,CO,Yuma County,8125963200,Transportation,NP,SLOW ELECTRICITY,CAP,NOx,Evolved,67.569863,67.569863,67.570059,1034.586811,0.065311,0.072115
31197,2017,CO,Yuma County,8125963200,Transportation,NP,SLOW ELECTRICITY,CAP,PM10,Evolved,3.245806,3.245806,3.245806,1034.586811,0.003137,0.071314
31198,2017,CO,Yuma County,8125963200,Transportation,NP,SLOW ELECTRICITY,CAP,PM2.5,Evolved,2.152553,2.152553,2.152556,1034.586811,0.002081,0.071314


In [21]:
drop_year_inds = subsector_emissions['Year'].isin(drop_years)
subsector_emissions = subsector_emissions[~drop_year_inds]

drop_year_inds = tract_emissions['Year'].isin(drop_years)
tract_emissions = tract_emissions[~drop_year_inds]

In [22]:
tract_emissions['Cumulative emissions (metric tons)'] = tract_emissions.groupby(['State','County',
                                                                                 'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].transform('cumsum')

tract_emissions['Percentage emission reduction (annual)'] = tract_emissions.groupby(['State','County',
                                                                                     'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].pct_change()

tract_emissions['Percentage emission reduction (cumulative)'] = tract_emissions.groupby(['State','County',
                                                                                         'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].apply(lambda x: x.div(x.iloc[0]).subtract(1))

tract_emissions['Emission reduction (annual)'] = tract_emissions.groupby(['State','County',
                                                                          'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].diff()

tract_emissions['Emission reduction (cumulative)'] = tract_emissions.groupby(['State','County',
                                                                              'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emission reduction (annual)'].cumsum()



In [23]:
tract_emissions_area = tract_emissions.join(tract_area, on='Census Tract')
emissions_cols = ['Emissions (tonnes / year)', 'Cumulative emissions (metric tons)',
       'Emission reduction (annual)', 'Emission reduction (cumulative)', 'Baseline emissions (tonnes / year)','Reference emissions (tonnes / year)']

for col in emissions_cols:
    tract_emissions_area[col + '_persqmi'] = tract_emissions_area[col] / tract_emissions_area['Tract Area (sqmi)']

In [24]:
tract_emissions_area['Baseline emissions (per sqmi) percentile'] = tract_emissions_area.groupby(['Scenario','Year','Pollutant Type'])['Baseline emissions (tonnes / year)_persqmi'].rank(pct=True)
tract_emissions_area.to_csv('../Evolved/my outputs/{}_transportation_tractEmissions_pctChange_UPDATED_{}.csv'.format(state, today_date), float_format='%.10f')

In [25]:
tract_emissions_area

,Year,State,County,Census Tract,Sector,Data Category,Scenario,Pollutant Category,Pollutant Type,Data source,Emissions (tonnes / year),Baseline emissions (tonnes / year),Reference emissions (tonnes / year),Cumulative emissions (metric tons),Percentage emission reduction (annual),Percentage emission reduction (cumulative),Emission reduction (annual),Emission reduction (cumulative),Tract Area (sqmi),Emissions (tonnes / year)_persqmi,Cumulative emissions (metric tons)_persqmi,Emission reduction (annual)_persqmi,Emission reduction (cumulative)_persqmi,Baseline emissions (tonnes / year)_persqmi,Reference emissions (tonnes / year)_persqmi,Baseline emissions (per sqmi) percentile
37350,2020,NM,Bernalillo County,35001000107,Transportation,NP,100 PERCENT RENEWABLES,CAP,CO,Evolved,19.388975,26.843504,19.435106,19.388975,NaN,0.000000,NaN,NaN,1.224806,15.830242,15.830242,NaN,NaN,21.916536,15.867905,0.510040
37351,2020,NM,Bernalillo County,35001000107,Transportation,NP,100 PERCENT RENEWABLES,CAP,NOx,Evolved,3.943253,6.283206,3.946291,3.943253,NaN,0.000000,NaN,NaN,1.224806,3.219492,3.219492,NaN,NaN,5.129960,3.221972,0.451807
37352,2020,NM,Bernalillo County,35001000107,Transportation,NP,100 PERCENT RENEWABLES,CAP,PM10,Evolved,0.309923,0.379167,0.310030,0.309923,NaN,0.000000,NaN,NaN,1.224806,0.253039,0.253039,NaN,NaN,0.309573,0.253126,0.467871
37353,2020,NM,Bernalillo County,35001000107,Transportation,NP,100 PERCENT RENEWABLES,CAP,PM2.5,Evolved,0.123053,0.190639,0.123150,0.123053,NaN,0.000000,NaN,NaN,1.224806,0.100467,0.100467,NaN,NaN,0.155649,0.100547,0.453815
37354,2020,NM,Bernalillo County,35001000107,Transportation,NP,100 PERCENT RENEWABLES,CAP (precursor),VOC,Evolved,1.602765,2.515857,1.606490,1.602765,NaN,0.000000,NaN,NaN,1.224806,1.308587,1.308587,NaN,NaN,2.054086,1.311628,0.508032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440725,2050,NM,Valencia County,35061971400,Transportation,NP,SLOW ELECTRICITY,CAP,NOx,Evolved,0.871585,167.821355,54.851223,964.240892,-0.292499,-0.991752,-0.360335,-104.794832,79.478622,0.010966,12.132079,-0.004534,-1.318529,2.111528,0.690138,0.307229
440726,2050,NM,Valencia County,35061971400,Transportation,NP,SLOW ELECTRICITY,CAP,PM10,Evolved,4.907463,8.766650,5.768467,150.149806,0.010056,-0.257633,0.048856,-1.703102,79.478622,0.061746,1.889185,0.000615,-0.021428,0.110302,0.072579,0.297189
440727,2050,NM,Valencia County,35061971400,Transportation,NP,SLOW ELECTRICITY,CAP,PM2.5,Evolved,0.665630,5.149512,1.595936,34.636505,0.002537,-0.783609,0.001684,-2.410430,79.478622,0.008375,0.435796,0.000021,-0.030328,0.064791,0.020080,0.305221
440728,2050,NM,Valencia County,35061971400,Transportation,NP,SLOW ELECTRICITY,CAP (precursor),VOC,Evolved,0.212619,39.998756,13.785912,241.680310,-0.288294,-0.991665,-0.086127,-25.297920,79.478622,0.002675,3.040822,-0.001084,-0.318298,0.503264,0.173454,0.285141


#### Aggregate medium and heavy-duty truck emissions.

#### Check CO demographic data for emissions pct increase.

In [12]:
ejscreen = pd.read_csv('../Demographic/Census Tract EJ Index Colorado.csv')
ejscreen.set_index(['TRACT'],inplace=True)
scen_inds = tract_emissions_area['Scenario']=='CENTRAL'
year_inds = tract_emissions_area['Year']==2030
pol_inds = tract_emissions_area['Pollutant Type']=='NOx'
central_year_pol = tract_emissions_area[scen_inds & year_inds & pol_inds]
central_year_pol_ejscreen = central_year_pol.join(ejscreen,on='Census Tract')
emissions_increase = central_year_pol_ejscreen['Percentage emission reduction (cumulative)'] > 0
over75_minority = central_year_pol_ejscreen['LessHSPtile'] > 0.75

### Aggregate NOx, PM2.5, and CO2 emissions at county level for year 2017.

In [21]:
county_emissions = pd.read_csv('../Evolved/my outputs/{}_transportation_tractEmissions_bySubsector_CAPsCO2_aug4.csv'.format(state))
county_emissions.drop(columns='Unnamed: 0', inplace=True)

In [22]:
county_emissions_2020 = county_emissions[county_emissions['Year']==2017]
county_emissions_ref2020 = county_emissions_2020[county_emissions_2020['Scenario']=='REFERENCE']
county_emissions_ref2020 = county_emissions_ref2020.groupby(['Year', 'State', 'County', 
        'Sector', 'Data Category', 'Scenario', 'Pollutant Category', 'Pollutant Type',
        'Data source'])['Emissions (tonnes / year)'].sum().reset_index()
pol_inds = county_emissions_ref2020['Pollutant Type'].isin(['NOx','PM2.5','CO2'])
county_emissions_ref2020 = county_emissions_ref2020[pol_inds]

In [24]:
county_emissions_ref2020.to_csv('../Evolved/my outputs/{}_transportation_countyEmissions_year2017_CO2NOxPM25_aug9.csv'.format(state))